This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# Load ODIM_H5 Volume data from German Weather Service

In this example, we obtain and read the latest 30 minutes of available volumetric radar data from German Weather Service available at [opendata.dwd.de](https://opendata.dwd.de). Finally we do some plotting.

This retrieves 6 timesteps of the 10 sweeps (moments DBZH and VRADH) of the DWD volume scan of a distinct radar. This amounts to 120 data files which are combined into one volumetric Cf/Radial2 like xarray powered structure.

Exports to single file Odim_H5 and Cf/Radial2 format are shown at the end of this tutorial.

<div class="alert alert-warning">

**Note** <br>

The following code is based on [xarray](https://docs.xarray.dev), [xarray-datatree](https://xarray-datatree.readthedocs.io) and [xradar](https://docs.openradarscience.org/projects/xradar). It claims multiple data files and presents them in a ``DataTree``.

</div>

In [ ]:
import wradlib as wrl
import warnings

warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xradar
from datatree import DataTree, open_datatree

try:
    get_ipython().run_line_magic("matplotlib inline")
except:
    plt.ion()

In [ ]:
import urllib3
import os
import io
import glob
import shutil
import datetime

In [ ]:
from html.parser import HTMLParser


class DWDHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        if tag != "a":
            return
        self.links.append(attrs[0][1])


parser = DWDHTMLParser()

# Download data from opendata.dwd.de

In [ ]:
radar = "ESS"
DBZH = "sweep_vol_z"
VRADH = "sweep_vol_v"

import certifi

opendata_url1 = f"https://opendata.dwd.de/weather/radar/sites/{DBZH}/{radar.lower()}/hdf5/filter_polarimetric/"

http = urllib3.PoolManager(cert_reqs="CERT_REQUIRED", ca_certs=certifi.where())
response = http.request("GET", opendata_url1).data.decode("utf-8")

parser.links = []
parser.feed(response)
filelist1 = parser.links[1:]

filelist1.sort(key=lambda x: x.split("-")[2])
filelist1.reverse()

opendata_url2 = f"https://opendata.dwd.de/weather/radar/sites/{VRADH}/{radar.lower()}/hdf5/filter_polarimetric/"

http = urllib3.PoolManager(cert_reqs="CERT_REQUIRED", ca_certs=certifi.where())
response = http.request("GET", opendata_url2).data.decode("utf-8")

parser.links = []
parser.feed(response)
filelist2 = parser.links[1:]

filelist2.sort(key=lambda x: x.split("-")[2])
filelist2.reverse()

## Clean up local folder

In [ ]:
flist = glob.glob("ras07*")
for f in flist:
    os.remove(f)

## Download latest 24 volumes to current directory

In [ ]:
for f in filelist1[: 10 * 24]:
    with http.request(
        "GET", os.path.join(opendata_url1, f), preload_content=False
    ) as r, open(f, "wb") as out:
        shutil.copyfileobj(r, out)

for f in filelist2[: 10 * 24]:
    with http.request(
        "GET", os.path.join(opendata_url2, f), preload_content=False
    ) as r, open(f, "wb") as out:
        shutil.copyfileobj(r, out)

In [ ]:
volume_reflectivity = glob.glob("ras07*_dbzh_*")
volume_velocity = glob.glob("ras07*_vradh_*")

In [ ]:
volume_reflectivity = np.array(
    sorted(volume_reflectivity, key=lambda x: x.split("-")[2])
)
volume_velocity = np.array(sorted(volume_velocity, key=lambda x: x.split("-")[2]))
volume_reflectivity[:15]

In [ ]:
volume_reflectivity = volume_reflectivity.reshape(-1, 10).T
volume_velocity = volume_velocity.reshape(-1, 10).T

In [ ]:
volume_reflectivity = sorted(
    volume_reflectivity, key=lambda x: x[0].split("-")[1].split("_")[-1]
)
volume_reflectivity = volume_reflectivity[5:] + volume_reflectivity[:5]
volume_velocity = sorted(
    volume_velocity, key=lambda x: x[0].split("-")[1].split("_")[-1]
)
volume_velocity = volume_velocity[5:] + volume_velocity[:5]
len(volume_reflectivity)

## Read the data into xarray powered structure

In [ ]:
dsl = []
reindex_angle = dict(
    tolerance=1.0, start_angle=0, stop_angle=360, angle_res=1.0, direction=1
)
for r, v in zip(volume_reflectivity, volume_velocity):
    ds0 = [
        xr.open_dataset(r0, engine="odim", group="sweep_0", reindex_angle=reindex_angle)
        for r0 in r
    ]
    ds1 = [
        xr.open_dataset(v0, engine="odim", group="sweep_0", reindex_angle=reindex_angle)
        for v0 in v
    ]
    ds = xr.concat(
        [
            xr.merge([r0, v0], compat="override").assign(volume_time=r0.time.min())
            for r0, v0 in zip(ds0, ds1)
        ],
        "volume_time",
    )
    dsl.append(ds)

dsl.insert(0, xr.open_dataset(volume_reflectivity[0][0], group="/"))

In [ ]:
tree = DataTree(data=xradar.io.backends.common._assign_root(dsl), name="root")
vol = xradar.io.backends.common._attach_sweep_groups(tree, dsl[1:])
for i, swp in enumerate(vol.groups[1:]):
    vol[swp]["sweep_number"] = i
# sets arbitrary volume_time in root group to enable isel over groups
vol = vol.assign_coords(volume_time=vol["sweep_0"].volume_time)

## Inspect structure
### Root Group

In [ ]:
vol.root

### Sweep Groups

In [ ]:
vol["sweep_0"]

## plot sweeps
### DBZH

In [ ]:
vol["sweep_0"].isel(volume_time=0)

In [ ]:
fig, gs = plt.subplots(
    4, 3, figsize=(20, 30), sharex=True, sharey=True, constrained_layout=True
)

for i, grp in enumerate(vol.groups[1:]):
    swp = vol[grp].isel(volume_time=0).ds
    swp = swp.assign_coords(sweep_mode=swp.sweep_mode)
    swp.DBZH.wrl.georef.georeference().wrl.vis.plot(ax=gs.flat[i], fig=fig)
    ax = plt.gca()
    ax.set_title(swp.sweep_fixed_angle.values)

fig.delaxes(gs.flat[-2])
fig.delaxes(gs.flat[-1])

### VRADH

In [ ]:
fig, gs = plt.subplots(
    4, 3, figsize=(20, 30), sharex=True, sharey=True, constrained_layout=True
)

for i, grp in enumerate(vol.groups[1:]):
    swp = vol[grp].isel(volume_time=0).ds
    swp = swp.assign_coords(sweep_mode=swp.sweep_mode)
    swp.VRADH.wrl.georef.georeference().wrl.vis.plot(ax=gs.flat[i], fig=fig)
    ax = plt.gca()
    ax.set_title(swp.sweep_fixed_angle.values)

fig.delaxes(gs.flat[-2])
fig.delaxes(gs.flat[-1])

### Plot single sweep using cartopy

In [ ]:
vol0 = vol.isel(volume_time=0)
swp = vol0["sweep_9"].ds
# need to assign sweep_mode as coordinate
swp = swp.assign_coords(sweep_mode=swp.sweep_mode)

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(
    central_latitude=vol0.root.ds.latitude.values,
    central_longitude=vol0.root.ds.longitude.values,
)

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=vol0.root.ds.latitude.values,
    central_longitude=vol0.root.ds.longitude.values,
)

pm = swp.DBZH.wrl.georef.georeference().wrl.vis.plot(crs=map_proj)
ax = plt.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=vol0.root.ds.longitude.values)
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp.DBZH.wrl.georef.georeference().wrl.vis.plot(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
fig = plt.figure(figsize=(10, 8))
proj = ccrs.AzimuthalEquidistant(
    central_latitude=vol0.root.ds.latitude.values,
    central_longitude=vol0.root.ds.longitude.values,
)
ax = fig.add_subplot(111, projection=proj)
pm = swp.DBZH.wrl.georef.georeference().wrl.vis.plot(ax=ax)
ax.gridlines()

### Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray inherits dimensions and coordinates of it's parent dataset. There are attributes connected which are defined by Cf/Radial and/or ODIM_H5 standard.

In [ ]:
vol["sweep_9"].isel(volume_time=0).ds.DBZH

In [ ]:
vol["sweep_9"].isel(volume_time=0).ds.sweep_mode

## Plot Quasi Vertical Profile

In [ ]:
ts = vol["sweep_9"]
ts

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(111)
ts.ds.DBZH.median("azimuth").plot(x="volume_time", vmin=-10, vmax=30, ax=ax)
ax.set_title(f"{np.datetime_as_string(ts.ds.time[0][0].values, unit='D')}")
ax.set_ylim(0, 20000)

## Export to OdimH5

This exports the radar volume at given timestep including all moments into one ODIM_H5 compliant data file.

In [ ]:
xradar.io.to_odim(vol0, "dwd_odim.h5", source="RAD:DWD")

## Export to Cf/Radial2

This exports the radar volume at given timestep including all moments into one Cf/Radial2 compliant data file.

In [ ]:
xradar.io.to_cfradial2(vol0, "dwd_cfradial2.nc")

## Import again


In [ ]:
vol1 = xradar.io.open_odim_datatree("dwd_odim.h5")
display(vol1)

In [ ]:
vol2 = open_datatree("dwd_cfradial2.nc")
display(vol2)